In [78]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [96]:
# with help from http://planspace.org/20150423-forward_selection_with_statsmodels/
import statsmodels.formula.api as smf

def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response, ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < (best_new_score * 1):
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response, ' + '.join(selected))
    
    model = smf.ols(formula, data).fit()
    return model

In [99]:
importedResponse=pd.read_pickle('data/ZipcodeMSPPS.pkl')
response=importedResponse[['RegionName','City','State','2000_agg','2010_agg','pct_delta']]
response['scaled_pct'] = response.pct_delta - 1
response=response.rename(columns={'RegionName': 'Zip_Code'})

/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [100]:
response.head(10)

,Zip_Code,City,State,2000_agg,2010_agg,pct_delta,scaled_pct
1,60629,Chicago,IL,98.682400,115.605500,1.171491,0.171491
2,90650,Norwalk,CA,130.505400,229.691000,1.760011,0.760011
7,90280,South Gate,CA,130.197000,212.896167,1.635185,0.635185
10,60618,Chicago,IL,151.892400,265.168433,1.745765,0.745765
13,91331,Los Angeles,CA,104.836333,195.163200,1.861599,0.861599
14,08701,Lakewood Township,NJ,74.834500,135.057367,1.804747,0.804747
15,92335,Fontana,CA,79.024267,118.977400,1.505581,0.505581
21,90805,Long Beach,CA,123.739133,222.963300,1.801882,0.801882
24,94565,Pittsburg,CA,117.949467,120.324433,1.020135,0.020135
27,92336,Fontana,CA,84.988300,127.604600,1.501437,0.501437


In [82]:
cdd=pd.read_pickle('data/cleaned_demographic_features.pkl')
cdd=cdd.rename(columns={'GEO.id2' : 'Zip_Code'})
print cdd.shape
cdd.head(10)

(33120, 59)


,GEO.id,Zip_Code,GEO.display-label,A001,A002,A003,A004,A005,A006,A007,A008,A009,A010,A011,A012,A013,A014,A015,A016,A017,A018,A019,A020,A021,A022,A023,A024,A025,A026,A027,A028,A029,A030,A031,A032,A033,A034,A035,A036,A037,A038,A039,A040,A041,A042,A043,A044,A045,A046,A047,A048,A049,A050,A051,A052,A053,A054,A055,A056
1,8600000US00601,00601,ZCTA5 00601,18570,12.8,16.2,13.9,11.7,13.3,13.1,10.2,5.7,3.1,48.9,6.6,8.5,6.9,5.4,6.3,6.3,4.9,2.7,1.3,51.1,6.3,7.7,7,6.3,6.9,6.8,5.3,2.9,1.8,93.1,3.1,0.4,0,0,99.5,77.8,5.1,2.6,20.3,9.6,20.3,10.2,3.3,10.1,5.5,39.5,29,84.3,15.7,2.9,0.5,2.84,3.27,7744,2.87
2,8600000US00602,00602,ZCTA5 00602,41520,11.5,15.2,13.5,12.9,14.6,14,10.1,5.7,2.3,49.1,5.9,7.7,6.7,6.4,7,6.8,4.9,2.6,1.1,50.9,5.6,7.5,6.8,6.5,7.6,7.2,5.2,3.1,1.3,86.7,5.3,0.3,0.1,0,99.4,77.6,5.1,2.1,17.4,8,20.5,10.5,3,10,4.9,37.6,25.9,83,17,2.6,1.3,2.76,3.20,18073,2.62
3,8600000US00603,00603,ZCTA5 00603,54689,12.2,14.2,13,13,12.7,12.6,12,7,3.3,48.6,6.4,7.3,6.5,6.4,6,5.8,5.6,3.2,1.4,51.4,5.8,7,6.5,6.6,6.6,6.7,6.4,3.7,2.1,82.9,7.6,0.3,0.2,0,98.5,71.1,4.7,1.7,22,10.9,26.4,12.7,4.1,13.8,6.8,34,31.1,82.5,17.5,3.6,1.5,2.53,3.06,25653,2.51
4,8600000US00606,00606,ZCTA5 00606,6615,12.8,15.1,14.7,12.1,13.1,13.4,10.5,5.5,2.9,49.4,6.6,7.5,7.1,6.1,6.5,6.3,5.2,2.6,1.4,50.6,6.2,7.5,7.6,6,6.6,7,5.4,2.9,1.5,88.9,4.7,0.4,0,0.1,99.4,74.6,6.2,2.7,18.4,8.1,22.8,12.7,4.5,10.1,4.7,37.5,28.4,83.6,16.4,1.7,0.8,2.75,3.24,2877,2.83
5,8600000US00610,00610,ZCTA5 00610,29016,11.8,15.1,12.6,13.4,13.9,12.6,11.2,6.7,2.8,48.6,6.2,7.7,6.4,6.6,6.5,5.9,5.1,3.2,1.1,51.4,5.6,7.5,6.2,6.9,7.4,6.7,6.1,3.5,1.7,82,7.2,0.3,0.1,0,99.2,74.4,4.9,1.9,19.9,9.4,22.8,11,3.4,11.9,5.8,36.5,29.4,85.9,14.1,1.9,0.8,2.67,3.14,12618,2.66
6,8600000US00612,00612,ZCTA5 00612,67010,11.4,14.3,12.5,12.6,13.1,12.2,11.8,7.7,4.2,47.4,6,7.3,6.2,6,6.2,5.5,5.2,3.4,1.7,52.6,5.5,7,6.4,6.7,6.9,6.6,6.7,4.4,2.5,84.3,6.2,0.4,0.1,0,99.1,71.7,4.8,1.8,22.4,10.2,25.6,10.7,3.8,14.8,7.8,33.8,33.7,84,16,3.1,2.2,2.55,3.05,30992,2.54
7,8600000US00616,00616,ZCTA5 00616,11017,11.8,14.9,11.8,13.3,14.5,11.4,11.9,6.6,3.7,48.3,6.1,7.5,5.9,6.1,6.9,5.2,5.5,3.2,1.6,51.7,5.7,7.3,5.9,7.2,7.6,6.2,6.4,3.4,2.1,84.8,7.1,0.3,0,0,99.5,73.3,4.4,1.5,20.5,9.1,24.3,12,5,12.2,6.2,36,31.6,84.6,15.4,2.3,1,2.65,3.16,4896,2.53
8,8600000US00617,00617,ZCTA5 00617,24597,13.4,15.1,13,15,12.7,10.9,10.6,6.1,3.2,48.1,6.8,7.8,6.3,7,5.9,5.1,5,2.8,1.3,51.9,6.5,7.4,6.7,8,6.9,5.7,5.5,3.3,1.8,80.6,7.7,0.3,0.1,0,99.4,74.7,5.2,2.2,23.6,11.5,22.8,11.4,3.9,11.4,5.5,38.9,29,85.7,14.3,1.9,2.5,2.71,3.18,10594,2.80
9,8600000US00622,00622,ZCTA5 00622,7853,9.6,13,11,11.1,12.5,13.1,14.6,9.7,5.3,48.2,4.9,6.5,5.2,5.3,6.4,6.1,7.1,4.6,2.1,51.8,4.7,6.5,5.8,5.8,6.2,7.1,7.6,5,3.2,85.8,5,0.2,0,0,98.2,69.1,5.5,2.6,15.5,5.9,26.9,13.4,4.2,13.4,7.3,28.9,37.8,36.6,63.4,3.7,4.9,2.45,2.97,8714,2.51
10,8600000US00623,00623,ZCTA5 00623,43061,12.3,14.4,11.3,14.1,13.3,11.3,11.7,7.5,4,47.8,6.4,7.3,5.6,6.6,6.4,5.2,5.4,3.5,1.6,52.2,6.1,7.1,5.7,7.6,6.9,6.1,6.3,4,2.3,83.8,5.5,0.3,0.1,0,99.1,71.6,4.9,2.2,19.3,9.4,25,11.5,4,13.5,7.3,35.7,32.6,77.6,22.4,1.8,2.7,2.58,3.08,21426,2.61


In [101]:
merged=pd.merge(response, cdd, on=['Zip_Code'])
merged=merged.drop(['Zip_Code','City','State','2000_agg','2010_agg','GEO.id','GEO.display-label'], 1)
merged.head(10)
merged=merged.astype('float')

In [ ]:
#minimerge = merged[0:50]
bestmodel = forward_selected(merged, 'scaled_pct')

In [98]:
bestmodel.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              pct_delta   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     4.000
Date:                Sun, 06 Dec 2015   Prob (F-statistic):           1.98e-05
Time:                        16:39:11   Log-Likelihood:                -3425.7
No. Observations:                1678   AIC:                             6873.
Df Residuals:                    1667   BIC:                             6933.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      4.0015      0.739      5.416      0.000         2.552     5.451
A052          -0.1509      0.063     -2.394      0.017        -0.274    -0.027
A013          -0.1324      0.045     -2.954      0.003        -0.220    -0.045
A031          -0.0138      0.005     -2.667      0.008        -0.024    -0.004
A041          -0.1316      0.057     -2.311      0.021        -0.243    -0.020
A034          -0.0145      0.009     -1.689      0.091        -0.031     0.002
A026          -0.7970      0.634     -1.256      0.209        -2.041     0.447
A030          -0.0702      0.040     -1.735      0.083        -0.150     0.009
A016          -1.0070      0.637     -1.582      0.114        -2.256     0.242
A006           0.9218      0.632      1.459      0.145        -0.317     2.161
A040           0.0365      0.035      1.039      0.299        -0.032     0.105
==============================================================================
Omnibus:                     4891.199   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        167269562.171
Skew:                          38.568   Prob(JB):                         0.00
Kurtosis:                    1547.818   Cond. No.                     1.88e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.88e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [10]:
response = 'pct_delta'
remaining = set(minimerge.columns)
remaining.remove(response)
formula = ''
for candidate in remaining:
    formula = formula + ' + ' + str(candidate)
formula = 'pct_delta ~ 1' + formula
formula

'pct_delta ~ 1 + A038 + A039 + A018 + A019 + A014 + A015 + A016 + A017 + A010 + A011 + A012 + A013 + A044 + A030 + A040 + A043 + A054 + A042 + A041 + A025 + A024 + A027 + A026 + A021 + A020 + A023 + A022 + A047 + A049 + A048 + A029 + A028 + A046 + A031 + A045 + A009 + A008 + A007 + A006 + A005 + A004 + A003 + A002 + A001 + A037 + A034 + A032 + A035 + A036 + A050 + A051 + A052 + A053 + A033 + A055 + A056'

In [76]:
import sklearn.linear_model as linear_model

Y = minimerge['pct_delta'].tolist()
X = np.array(minimerge.drop('pct_delta',1))

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
model = regr.fit(X, Y)

In [72]:
import sklearn.linear_model as linear_model
from sklearn.cross_validation import cross_val_predict

Y = minimerge['pct_delta'].tolist()
X = np.array(minimerge.drop('pct_delta',1))

# Create linear regression object
lr = linear_model.LinearRegression()

# Train the model using the training sets
predicted = cross_val_predict(lr, X, Y, cv=10)

In [75]:
dir(lr)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_center_data',
 '_get_param_names',
 '_set_intercept',
 'copy_X',
 'decision_function',
 'fit',
 'fit_intercept',
 'get_params',
 'n_jobs',
 'normalize',
 'predict',
 'score',
 'set_params']

In [61]:
regmodel.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              pct_delta   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                     0.000
Date:                Sun, 06 Dec 2015   Prob (F-statistic):                nan
Time:                        15:55:39   Log-Likelihood:                 1410.1
No. Observations:                  50   AIC:                            -2720.
Df Residuals:                       0   BIC:                            -2625.
Df Model:                          49                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------------------
Intercept                      0.0101        inf          0        nan           nan       nan
A038[T.3.8]                    0.0026        inf          0        nan           nan       nan
A038[T.4.0]                    0.0045        inf          0        nan           nan       nan
A038[T.4.5]                    0.0149        inf          0        nan           nan       nan
A038[T.4.7]                   -0.0073        inf         -0        nan           nan       nan
A038[T.5.1]                    0.0039        inf          0        nan           nan       nan
A038[T.5.2]                    0.0055        inf          0        nan           nan       nan
A038[T.5.5]                   -0.0107        inf         -0        nan           nan       nan
A038[T.5.7]                   -0.0052        inf         -0        nan           nan       nan
A038[T.5.8]                   -0.0035        inf         -0        nan           nan       nan
A038[T.6.2]                    0.0050        inf          0        nan           nan       nan
A038[T.6.3]                   -0.0026        inf         -0        nan           nan       nan
A038[T.6.5]                   -0.0046        inf         -0        nan           nan       nan
A038[T.6.8]                    0.0091        inf          0        nan           nan       nan
A038[T.6.9]                    0.0210        inf          0        nan           nan       nan
A038[T.7.0]                    0.0097        inf          0        nan           nan       nan
A038[T.7.2]                   -0.0021        inf         -0        nan           nan       nan
A038[T.7.3]                   -0.0031        inf         -0        nan           nan       nan
A038[T.7.4]                   -0.0026        inf         -0        nan           nan       nan
A038[T.7.6]                   -0.0047        inf         -0        nan           nan       nan
A038[T.8.1]                   -0.0047        inf         -0        nan           nan       nan
A038[T.8.2]                    0.0025        inf          0        nan           nan       nan
A038[T.8.4]                   -0.0154        inf         -0        nan           nan       nan
A038[T.8.5]                    0.0005        inf          0        nan           nan       nan
A038[T.8.6]                    0.0003        inf          0        nan           nan       nan
A038[T.8.7]                   -0.0122        inf         -0        nan           nan       nan
A038[T.8.8]                   -0.0007        inf         -0        nan           nan       nan
A038[T.9.0]                    0.0049        inf          0        nan           nan       nan
A038[T.9.4]                    0.0026        inf          0        nan           nan       nan
A038[T.9.5]                    0.0016      